# Feature Transformation and Selection

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load all sheets from the Excel file
file_path = '../data/DataScientist_CaseStudy_Dataset.xlsx'

# Load sheets
soc_dem = pd.read_excel(file_path, sheet_name='Soc_Dem')
print(f"Shape of soc_dem: {soc_dem.shape}")
products = pd.read_excel(file_path, sheet_name='Products_ActBalance')
print(f"Shape of products: {products.shape}")
inflow_outflow = pd.read_excel(file_path, sheet_name='Inflow_Outflow')
print(f"Shape of inflow_outflow: {inflow_outflow.shape}")
sales_revenue = pd.read_excel(file_path, sheet_name='Sales_Revenues')
print(f"Shape of sales_revenue: {sales_revenue.shape}")
df = sales_revenue.merge(products, on='Client', how='left')\
            .merge(inflow_outflow, on='Client', how='left')\
               .merge(soc_dem, on='Client', how='left')

print(df.shape)
print("Duplicated: ", df.duplicated(subset= "Client").sum())
label_col = ['Sale_MF', 'Sale_CC', 'Sale_CL', 'Revenue_MF', 'Revenue_CC', 'Revenue_CL']
feature_col = [col for col in df.columns if col not in label_col + ['Client']]

df.head()


Shape of soc_dem: (1615, 4)
Shape of products: (1615, 13)
Shape of inflow_outflow: (1587, 15)
Shape of sales_revenue: (969, 7)
(969, 36)
Duplicated:  0


,Client,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Count_CA,Count_SA,Count_MF,...,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sex,Age,Tenure
0,1094,0,1,0,0.000000,5.138571,0.0000,1,NaN,NaN,...,351.950357,891.327857,31.0,31.0,1.0,21.0,4.0,M,25,8
1,1484,1,0,1,25.609107,0.000000,8.7225,1,NaN,NaN,...,302.446429,357.785714,23.0,23.0,1.0,16.0,5.0,M,39,0
2,20,0,1,0,0.000000,3.822500,0.0000,1,1.0,NaN,...,10.714286,103.571429,7.0,7.0,3.0,1.0,3.0,M,27,181
3,1193,0,0,0,0.000000,0.000000,0.0000,2,NaN,NaN,...,331.150000,75.178571,55.0,36.0,6.0,21.0,2.0,M,48,16
4,142,0,0,0,0.000000,0.000000,0.0000,1,NaN,NaN,...,0.000000,725.928571,2.0,2.0,0.0,0.0,1.0,F,27,183


In [3]:
feature_col

['Count_CA',
 'Count_SA',
 'Count_MF',
 'Count_OVD',
 'Count_CC',
 'Count_CL',
 'ActBal_CA',
 'ActBal_SA',
 'ActBal_MF',
 'ActBal_OVD',
 'ActBal_CC',
 'ActBal_CL',
 'VolumeCred',
 'VolumeCred_CA',
 'TransactionsCred',
 'TransactionsCred_CA',
 'VolumeDeb',
 'VolumeDeb_CA',
 'VolumeDebCash_Card',
 'VolumeDebCashless_Card',
 'VolumeDeb_PaymentOrder',
 'TransactionsDeb',
 'TransactionsDeb_CA',
 'TransactionsDebCash_Card',
 'TransactionsDebCashless_Card',
 'TransactionsDeb_PaymentOrder',
 'Sex',
 'Age',
 'Tenure']

In [4]:
# Fill missing
col_exclude = 'Sex'

df[df.columns.difference([col_exclude])] = df[df.columns.difference([col_exclude])].fillna(0)
df = df.dropna() # Dropna
df.isnull().sum()

Client                          0
Sale_MF                         0
Sale_CC                         0
Sale_CL                         0
Revenue_MF                      0
Revenue_CC                      0
Revenue_CL                      0
Count_CA                        0
Count_SA                        0
Count_MF                        0
Count_OVD                       0
Count_CC                        0
Count_CL                        0
ActBal_CA                       0
ActBal_SA                       0
ActBal_MF                       0
ActBal_OVD                      0
ActBal_CC                       0
ActBal_CL                       0
VolumeCred                      0
VolumeCred_CA                   0
TransactionsCred                0
TransactionsCred_CA             0
VolumeDeb                       0
VolumeDeb_CA                    0
VolumeDebCash_Card              0
VolumeDebCashless_Card          0
VolumeDeb_PaymentOrder          0
TransactionsDeb                 0
TransactionsDe

In [5]:
# Filter erroneous tenure
print(f"Pre-filtered shape: {len(df)}")
df =  df[df['Tenure']/12 <= df['Age']]
print(f"Post-filtered shape: {len(df)}")

Pre-filtered shape: 967
Post-filtered shape: 933


In [6]:
# Create Dummy Columns
dummies = pd.get_dummies(df['Sex'], prefix='Sex', drop_first=True).astype(int)
df = df.drop('Sex', axis=1).join(dummies)

In [7]:
# Log transform
col_to_transform = ["Age", 'Tenure'] + [col for col in df.columns if col.startswith('Volume') or col.startswith('ActBal') or col.startswith('Transactions')]
df[col_to_transform] = df[col_to_transform].apply(lambda x: np.log1p(x))
df.describe()

c:\Anaconda\envs\myenv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Client,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,Count_CA,Count_SA,Count_MF,...,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Age,Tenure,Sex_M
count,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,...,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000,933.000000
mean,809.939979,0.202572,0.250804,0.286174,1.984476,2.783249,3.388565,1.077170,0.319400,0.926045,...,2.751526,4.164421,2.357480,2.247337,0.790142,1.117071,1.280376,3.728836,4.230763,0.535906
std,463.643784,0.402132,0.433708,0.452214,10.123181,18.021070,7.554915,0.328287,0.577492,4.124133,...,2.627080,2.804534,1.082752,1.030170,0.759141,1.177716,0.971236,0.434789,1.076944,0.498977
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000
25%,420.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.609438,1.609438,0.000000,0.000000,0.000000,3.433987,3.713572,0.000000
50%,810.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,3.241476,5.162539,2.564949,2.397895,0.693147,0.693147,1.386294,3.761200,4.532599,1.000000
75%,1212.000000,0.000000,1.000000,1.000000,0.000000,0.071429,4.214286,1.000000,1.000000,0.000000,...,5.200303,6.196953,3.135494,2.995732,1.386294,2.079442,2.079442,4.077537,5.023881,1.000000
max,1613.000000,1.000000,1.000000,1.000000,220.488214,407.962500,133.275357,4.000000,3.000000,79.000000,...,8.199359,11.034181,4.634729,4.430817,3.258097,4.110874,3.555348,4.510860,5.605802,1.000000


# Train Test Split

In [13]:
from sklearn.model_selection import train_test_split

def split_df(df, label_cols, test_size=0.1, val_size=0.1, random_state=None, shuffle=True):
    """
    Split a DataFrame into train, validation, and test sets with stratification
    based on the combination of multiple label columns.

    Parameters:
    - df: pandas DataFrame
    - label_cols: list of str, column names for labels (for stratification)
    - test_size: float, fraction of data for test set (default 0.2)
    - val_size: float, fraction of data for validation set (default 0.1)
    - random_state: int or None, for reproducibility
    - shuffle: bool, whether to shuffle before splitting (default True)

    Returns:
    - X_train, X_val, X_test, y_train, y_val, y_test as DataFrames
    """
    if not isinstance(label_cols, list):
        raise ValueError("label_cols must be a list of column names")

    # Create stratification key by combining label columns as a tuple of string
    stratify_key = df[label_cols].astype(str).agg('_'.join, axis=1)

    X = df.drop(columns=label_cols)
    y = df[label_cols]

    # First split: train+val and test with stratification
    X_train_val, X_test, y_train_val, y_test, strat_train_val, strat_test = train_test_split(
        X, y, stratify_key, test_size=test_size, random_state=random_state, shuffle=shuffle
    )

    # Adjust val_size relative to remaining train_val set
    val_adjusted = val_size / (1 - test_size)

    # Second split: train and val with stratification
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_adjusted,
        stratify=strat_train_val,
        random_state=random_state, shuffle=shuffle
    )

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = split_df(
    df, label_cols=['Sale_MF', 'Sale_CC', 'Sale_CL'], test_size=0.1, val_size=0.1, random_state=42
)
print(X_train.shape, X_test.shape, X_val.shape)
X_train.head()

(745, 33) (94, 33) (94, 33)


,Client,Revenue_MF,Revenue_CC,Revenue_CL,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Age,Tenure,Sex_M
194,721,0.0,4.464286,0.000000,1,1.0,0.0,1.0,0.0,0.0,...,5.030134,6.703035,2.890372,2.890372,0.693147,1.609438,2.484907,4.219508,4.418841,0
120,1456,0.0,3.607143,0.000000,1,1.0,0.0,0.0,0.0,0.0,...,6.211492,5.559169,3.367296,3.367296,0.693147,3.044522,1.791759,4.127134,5.010635,0
859,1579,0.0,0.000000,4.388929,1,1.0,3.0,0.0,0.0,0.0,...,6.819962,6.669770,3.258097,3.258097,1.791759,2.397895,2.197225,3.931826,0.000000,1
705,1272,0.0,0.000000,0.000000,1,0.0,1.0,0.0,0.0,0.0,...,5.667504,6.830141,2.944439,2.944439,1.386294,1.609438,2.302585,3.583519,4.574711,1
762,123,0.0,0.000000,10.461071,2,0.0,0.0,0.0,0.0,0.0,...,3.553305,0.000000,1.098612,1.098612,0.000000,0.693147,0.000000,3.135494,3.713572,0


# Modelling and Analysis

In [ ]:
import numpy as np
import pandas as pd
import yaml
from typing import List, Optional, Dict, Any, Union
from pydantic import BaseModel, Field, validator
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    f1_score, precision_score, recall_score, roc_auc_score, classification_report
)
from sklearn.model_selection import GridSearchCV
import wandb
import warnings

warnings.filterwarnings("ignore")


class ModelHyperparams(BaseModel):
    random_forest: Optional[Dict[str, Any]] = Field(default_factory=dict)
    xgboost: Optional[Dict[str, Any]] = Field(default_factory=dict)


class WandbConfig(BaseModel):
    project_name: str
    run_name: Optional[str] = None
    hyperparameters: ModelHyperparams


class ClassifierTrainer:
    def __init__(
        self,
        model_name: str,
        config: ModelHyperparams,
        feature_cols: List[str],
        label_col: str,
        wandb_config: WandbConfig,
    ):
        """
        Initialize the Classifier Trainer with W&B tracking.

        Parameters:
        - model_name: 'random_forest' or 'xgboost'
        - config: ModelHyperparams object containing hyperparameters to tune per model
        - feature_cols: list of feature column names
        - label_col: name of label column
        - wandb_config: WandbConfig object with wandb settings
        """
        self.model_name = model_name
        self.config = config
        self.feature_cols = feature_cols
        self.label_col = label_col
        self.best_threshold = 0.5
        self.model = self._init_model()

        # Start WandB run
        self.wandb_run = wandb.init(
            project=wandb_config.project_name,
            name=wandb_config.run_name,
            config=config.dict(),
        )
        wandb.config.update({"model_name": model_name})

    def _init_model(self):
        if self.model_name == "random_forest":
            return RandomForestClassifier(random_state=42)
        elif self.model_name == "xgboost":
            return XGBClassifier(
                use_label_encoder=False,
                eval_metric="logloss",
                random_state=42,
                n_estimators=1000,
                learning_rate=0.001,
            )
        else:
            raise ValueError(f"Unsupported model_name: {self.model_name}")

    def tune_hyperparameters(self, train_df: pd.DataFrame, val_df: pd.DataFrame):
        """
        Hyperparameter tuning with GridSearchCV and threshold optimization on validation set.
        """
        X_train, y_train = train_df[self.feature_cols], train_df[self.label_col]
        X_val, y_val = val_df[self.feature_cols], val_df[self.label_col]

        param_grid = getattr(self.config, self.model_name, {})
        if not param_grid:
            raise ValueError(
                f"No hyperparameter grid provided for {self.model_name} in config."
            )

        grid_search = GridSearchCV(
            self.model, param_grid, scoring="f1", cv=3, n_jobs=-1, verbose=1
        )
        grid_search.fit(X_train, y_train)
        self.model = grid_search.best_estimator_

        wandb.log({"best_hyperparameters": grid_search.best_params_})
        print(f"Best hyperparameters for {self.model_name}: {grid_search.best_params_}")

        val_probs = self.model.predict_proba(X_val)[:, 1]
        self.best_threshold = self._tune_threshold(y_val.values, val_probs)
        wandb.log({"best_threshold": self.best_threshold})
        print(f"Best threshold tuned on validation set: {self.best_threshold:.3f}")

    def _tune_threshold(self, y_true: np.ndarray, y_probs: np.ndarray) -> float:
        """
        Find the optimal threshold to maximize F1 score.
        """
        thresholds = np.linspace(0.1, 0.9, 81)
        f1_scores = []
        for t in thresholds:
            preds = (y_probs >= t).astype(int)
            f1 = f1_score(y_true, preds)
            f1_scores.append(f1)
        best_idx = np.argmax(f1_scores)
        return thresholds[best_idx]

    def evaluate(self, test_df: pd.DataFrame) -> Dict[str, float]:
        """
        Evaluate on test data with custom threshold.
        """
        X_test = test_df[self.feature_cols]
        y_test = test_df[self.label_col]

        probs = self.model.predict_proba(X_test)[:, 1]
        preds = (probs >= self.best_threshold).astype(int)

        metrics = {
            "f1": f1_score(y_test, preds),
            "precision": precision_score(y_test, preds, zero_division=0),
            "recall": recall_score(y_test, preds),
            "auc": roc_auc_score(y_test, probs),
            "threshold": self.best_threshold,
        }

        wandb.log(metrics)
        return metrics

    def predict(self, df: pd.DataFrame) -> (np.ndarray, np.ndarray):
        """
        Predict labels and probabilities.
        """
        X = df[self.feature_cols]
        probs = self.model.predict_proba(X)[:, 1]
        preds = (probs >= self.best_threshold).astype(int)
        return preds, probs

    def close(self):
        """
        Finish WandB run cleanly.
        """
        wandb.finish()


def full_model_evaluation(
    model,
    X_test: Union[pd.DataFrame, np.ndarray],
    y_test: Union[pd.Series, np.ndarray],
    top_percent: float = 0.15,
    positive_label: int = 1,
    log_to_wandb: bool = True,
) -> Dict[str, float]:
    """
    Comprehensive evaluation with W&B logging.
    """
    if hasattr(model, "predict_proba"):
        y_scores = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        y_scores = model.decision_function(X_test)
    else:
        raise ValueError("Model must support predict_proba or decision_function")

    auc = roc_auc_score(y_test, y_scores)
    y_pred_05 = (y_scores >= 0.5).astype(int)

    precision = precision_score(y_test, y_pred_05, zero_division=0)
    recall = recall_score(y_test, y_pred_05)
    f1 = f1_score(y_test, y_pred_05)

    report = classification_report(y_test, y_pred_05, output_dict=True)
    minority_report = report.get(str(positive_label), {})

    # Top X% targeting
    n_target = max(1, int(len(y_scores) * top_percent))
    sorted_indices = np.argsort(y_scores)[::-1]
    target_indices = sorted_indices[:n_target]

    y_target = y_test[target_indices]
    precision_at_top = np.mean(y_target == positive_label)
    total_positives = np.sum(y_test == positive_label)
    recall_at_top = (
        np.sum(y_target == positive_label) / total_positives if total_positives > 0 else 0
    )

    results = {
        "AUC": auc,
        "Precision_0.5": precision,
        "Recall_0.5": recall,
        "F1_0.5": f1,
        "Precision_at_top_percent": precision_at_top,
        "Recall_at_top_percent": recall_at_top,
        "Total_Targeted": n_target,
        "Total_Positives": total_positives,
    }

    if log_to_wandb:
        wandb.log(results)
        if minority_report:
            wandb.log(
                {
                    "Minority_Precision": minority_report.get("precision", 0),
                    "Minority_Recall": minority_report.get("recall", 0),
                    "Minority_F1": minority_report.get("f1-score", 0),
                    "Minority_Support": minority_report.get("support", 0),
                }
            )

    return results


def analyze_feature_importance(
    model,
    X_test: Union[pd.DataFrame, np.ndarray],
    feature_names: Optional[List[str]] = None,
    top_n: Optional[int] = None,
    plot: bool = True,
) -> pd.DataFrame:
    """
    Analyze and rank feature importance for Random Forest or XGBoost.

    Parameters:
    - model: Trained model (RandomForestClassifier or XGBClassifier)
    - X_test: Test dataset features (DataFrame or array)
    - feature_names: Optional list of feature names (if X_test is array)
    - top_n: Show top N most relevant features (None = show all)
    - plot: Whether to show bar plot of feature importance

    Returns:
    - Pandas DataFrame with columns: ["feature", "importance"], sorted descending
    """

    import matplotlib.pyplot as plt
    import seaborn as sns

    if feature_names is None:
        if isinstance(X_test, pd.DataFrame):
            feature_names = X_test.columns.tolist()
        else:
            raise ValueError("Provide feature_names if X_test is not a DataFrame")

    # Get importance scores
    if hasattr(model, "feature_importances_"):
        importance = model.feature_importances_
    elif hasattr(model, "get_booster"):  # For XGBoost native booster
        importance_dict = model.get_booster().get_score(importance_type="gain")
        importance = np.array(
            [importance_dict.get(f"f{i}", 0.0) for i in range(len(feature_names))]
        )
    else:
        raise ValueError("Model does not provide feature importance")

    importance_df = (
        pd.DataFrame({"feature": feature_names, "importance": importance})
        .sort_values(by="importance", ascending=False)
        .reset_index(drop=True)
    )

    if top_n:
        importance_df = importance_df.head(top_n)

    if plot:
        plt.figure(figsize=(10, max(4, 0.4 * len(importance_df))))
        sns.barplot(x="importance", y="feature", data=importance_df, palette="viridis")
        plt.title("Feature Importance (Most to Least Relevant)")
        plt.show()

    return importance_df


def load_wandb_config(yaml_path: str) -> WandbConfig:
    """
    Load wandb config from yaml and validate with Pydantic.
    """
    with open(yaml_path, "r") as f:
        cfg = yaml.safe_load(f)
    return WandbConfig.parse_obj(cfg)

## MF

In [ ]:
config = {
    'random_forest': {
        'max_depth': [5, 10],
        'min_samples_split': [2, 5],
    },
    'xgboost': {
        'reg_alpha': [1, 10],
        'reg_lambda': [2, 10],
        
    }
}

In [ ]:
print(X_train.columns)
feature_cols = ['ActBal_CA', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC',
       'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred',
       'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder', 'Age', 'Tenure', 'Sex_M']  

X_train_filtered = X_train[feature_cols]
X_test_filtered = X_test[feature_cols]
X_val_filtered = X_val[feature_cols]

y_train_MF, y_test_MF, y_val_MF = y_train["Sale_MF"], y_test["Sale_MF"], y_val["Sale_MF"]
y_train_CC, y_test_CC, y_val_CC = y_train["Sale_CC"], y_test["Sale_CC"], y_val["Sale_CC"]
y_train_CL, y_test_CL, y_val_CL = y_train["Sale_CL"], y_test["Sale_CL"], y_val["Sale_CL"]  

In [ ]:

# Load your wandb config from yaml
wandb_config = load_wandb_config("config.yaml")

# Initialize trainer
trainer = ClassifierTrainer(
    model_name="random_forest",  # or 'xgboost'
    config=wandb_config.hyperparameters,
    feature_cols=feature_cols,
    label_col=label_col,
    wandb_config=wandb_config,
)

# Tune hyperparameters
trainer.tune_hyperparameters(train_df, val_df)

# Evaluate on test set
metrics = trainer.evaluate(test_df)
print(metrics)

# Optionally analyze feature importance
importance_df = analyze_feature_importance(trainer.model, test_df[feature_cols])
print(importance_df)

# Close wandb run
trainer.close()

## CC

In [ ]:
# Random Forest

trainer = ClassifierTrainer('random_forest', config, feature_cols, label_col)
trainer.tune_hyperparameters(train_df, val_df)
metrics = trainer.evaluate(test_df)
print(metrics)

In [ ]:
# Xgboost



## CL

# Optimization

In [ ]:
# Which clients have a higher propensity to buy a consumer loan?

In [ ]:
# Which clients have a higher propensity to buy a credit card?

In [ ]:
# Which clients have a higher propensity to buy a mutual fund?

In [ ]:
# What would be the expected revenue based on your strategy (median per customer)?